# Create demo wbia database to test re-identification of whale sharks

In [1]:
import matplotlib.pyplot as plt
import os
import shutil
import imageio
import json
import numpy as np

%matplotlib inline

## Create json file for demo db

Demo database consists of whale sharks with number of sightings per name from 2 to 5. Thus every query image have maximum of 4 matches.

Download dataset of whale sharks and extract it into the directory `wbia_pie_v2/reid-data/whaleshark_cropped/original`. Specify json file with test annotations:

Train/test annotations in json format are saved during data preprocessing. Load test annotations:

In [2]:
species = 'graywhale'
dataDir = '../reid-data/{}/processed'.format(species)
dataType= 'test_left'
imageDir = '{}/images/{}'.format(dataDir, dataType)
annFile='{}/annots/{}.json'.format(dataDir,dataType)
targetType = 'test2021'

Initialize a directory for a demo db:

In [3]:
target_dir = '../reid-data/{}/demo_db'.format(species)
targer_image_dir = os.path.join(target_dir, 'images', targetType)
targer_annot_dir = os.path.join(target_dir, 'annotations')

os.makedirs(targer_annot_dir, exist_ok=True)
os.makedirs(targer_image_dir, exist_ok=True)

Read COCO annotations:

In [4]:
with open(annFile, 'r') as f:
    test_annots = json.load(f)
print('Found {} annotations'.format(len(test_annots)))

Found 3467 annotations


Select test names with number of annotations per name equal to 3 or more.

In [5]:
test_names_all = np.array([a['name'] for a in test_annots])

In [6]:
unames, counts = np.unique(test_names_all, return_counts=True)
test_names = unames[counts>=3]
print('Found {} test names with 3 or more examples'.format(len(test_names)))

Found 359 test names with 3 or more examples


In [7]:
test_annots[0]

{'image_path': '/home/olga/projects/wbia-plugin-pie-v2/wbia_pie_v2/reid-data/graywhale/processed/images/test_left/000000000002_0.jpg',
 'aa_bbox': [0, 0, 1479, 256],
 'obj_id': 0,
 'name': '111f3384-3874-410d-8ca9-a200f2d42559',
 'viewpoint': 'left',
 'id_name': '111f3384-3874-410d-8ca9-a200f2d42559_left'}

In [8]:
annotations = []
images = []

for idx, record in enumerate(test_annots):
    image_path = record['image_path']
    name = record['name']
    viewpoint = record['viewpoint']
    source_basename = os.path.split(image_path)[-1]
    target_path = os.path.join(targer_image_dir, source_basename)
    shutil.copyfile(image_path, target_path)
    # these images are pre-cropped aka trivial annotations
    h, w = imageio.imread(image_path).shape[:2]
    image = {'license': 1,
             'file_name': source_basename,
             'photographer': '',
             'coco_url': None,
             'height': h,
             'width': w,
             'date_captured': 'NA',
             'gps_lat_captured': '-1.000000',
             'gps_lon_captured': '-1.000000',
             'flickr_url': None,
             'id': idx}
    annot = {'bbox': [0, 0, w, h],
              'theta': 0.0,
              'viewpoint': viewpoint,
              'segmentation': [[0, 0, h, 0, h, w, 0, w, 0, 0]],
              'segmentation_bbox': [0, 0, w, h],
              'area': h*w,
              'iscrowd': 0,
              'id': idx,
              'image_id': idx,
              'category_id': 0,
              'individual_ids': [],
              'isinterest': 0,
              'name': name}
    
    if name in test_names:
        images.append(image)
        annotations.append(annot)

In [9]:
print('Saved {} images and {} annotations'.format(len(images), len(annotations)))

Saved 1879 images and 1879 annotations


Load original COCO annotations to copy data such as `info`, `categories` and `licences`.

In [10]:
annOriginalFile = '../reid-data/{}/original/{}.coco/annotations/instances_{}.json'.format(species, species, 'test2021')

In [11]:
with open(annOriginalFile, 'r') as f:
    original_annots = json.load(f)

In [12]:
coco_annotations = {
                    'info': original_annots['info'],
                    'licenses': original_annots['licenses'],
                    'categories': original_annots['categories'],
                    'images': images,
                    'annotations': annotations
                   }

Save COCO annotations for demo database:

In [13]:
annot_filename = os.path.join(targer_annot_dir, 'instances_{}.json'.format(targetType))
with open(annot_filename, 'w') as f:
    json.dump(coco_annotations, f, indent=4, separators=(',', ': '))

In [14]:
shutil.make_archive('pie_v2.whale_grey_demo', 'zip', root_dir=target_dir)

'/home/olga/projects/wbia-plugin-pie-v2/wbia_pie_v2/notebooks-data-analysis/pie_v2.whale_grey_demo.zip'

Upload the created demo database archive to a public server for automated testing with plugin.